# Humanoid Env

In [25]:
import gymnasium as gym
import numpy as np

In [26]:
# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# To get smooth animations
import matplotlib.animation as animation
mpl.rc('animation', html='jshtml')

In [27]:
# Crear el entorno Humanoid
env = gym.make("Humanoid-v4", render_mode='human')

## Q-learning

In [28]:
# Inicializar la tabla Q con valores iniciales

env.reset() # reset the environment to a new random state

(array([ 1.40089773e+00,  9.99897788e-01, -7.66129031e-03,  7.48330749e-03,
        -9.47194206e-03,  6.09659749e-03, -3.90875665e-03,  8.27559572e-03,
        -1.89389489e-03,  5.85556405e-03,  1.24976935e-03,  6.33932036e-03,
        -5.42981506e-03, -5.12888324e-03,  9.03139958e-03, -4.62746199e-03,
         2.75643350e-03, -6.60397168e-03, -6.57194243e-03,  4.57250598e-03,
         1.77490595e-03,  5.17077391e-03,  9.05367625e-03,  5.48874419e-03,
         8.39544396e-04,  2.02187204e-03,  3.54659908e-03,  1.75778583e-03,
        -4.38448080e-03,  4.99472649e-03,  9.35861584e-03,  8.59264612e-03,
         4.31193807e-03, -9.23030681e-03, -9.25550563e-03,  1.73147902e-03,
         7.40642682e-03, -8.34437842e-03,  3.38145327e-03, -8.12498102e-03,
        -7.08681191e-03,  6.72073070e-03, -5.24704762e-03,  5.52260102e-03,
         5.28915642e-04,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.0

In [29]:
print("Action Space {}". format(env.action_space))
print("State Space {}.". format(env.observation_space))

action_space_size = env.action_space.shape[0]
state_space_size = env.observation_space.shape[0]
q_table = np.zeros((state_space_size, action_space_size))

Action Space Box(-0.4, 0.4, (17,), float32)
State Space Box(-inf, inf, (376,), float64).


In [30]:
q_table.shape

(376, 17)

In [31]:
# Definir parámetros
alpha = 0.1  # Tasa de aprendizaje
gamma = 0.99  # Factor de descuento
epsilon = 0.1  # Probabilidad de exploración inicial
num_episodes = 1000

In [32]:
# Función para elegir una acción basada en la política epsilon-greedy
def choose_action(state):
    state_index = hash(tuple(state))
    if np.random.uniform(0, 1) < epsilon:
        return env.action_space.sample()  # Exploración aleatoria
    else:
        return np.argmax(q_table[state_index, :])  # Explotación

In [36]:
state[np.newaxis]

TypeError: tuple indices must be integers or slices, not NoneType

In [34]:
# Bucle de entrenamiento
for episode in range(num_episodes):
    state = env.reset()
    state2 = state[0]

    epochs, penalties, reward, = 0, 0, 0
    done = False

    while not done:
        # Elegir una acción
        action = np.random.randn(env.action_space.shape[0],1)
        action = choose_action(state2)
        # Tomar la acción y observar el siguiente estado y la recompensa
        next_state, reward, done, truncate, info = env.step(action) 
  
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
  
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        #print(state[0], action, new_value)
        q_table[state,action] = new_value
        #print(state[0], action, new_value)
        #print(reward)
        
        if reward == -10:
            penalties += 1

        state= next_state
        epochs += 1
        #print(state)
    if episode % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {episode}")

print("Entrenamiento completado.")

IndexError: index 1485138634596624568 is out of bounds for axis 0 with size 376